In [1]:
import sqlite3
import pandas as pd
import numpy as np
import urllib
import json

In [2]:
# 시 코드 & SGG → 각 도시별 시군구 데이터 
# SGG, EMD
def getAddInfo(code, types):
    res = urllib.request.urlopen("http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?bjd_code={}&bjd_gbn={}".format(code,types))
    data = json.loads(res.read().decode('utf-8'))
    df = pd.DataFrame(data['resultList'])
    df['CODE2'] = str(code)
    return df

# 아파트코드마다 아파트 데이터 얻기
def getAptInfo(kapt_code):
    res = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code={}".format(kapt_code))
    data = json.loads(res.read().decode('utf-8'))
    return data

In [3]:
def getAptInfo2(kapt_code):
    res = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code={}".format(kapt_code))
    data = json.loads(res.read().decode('utf-8'))
    동수 = int(data['resultMap_kapt']['KAPT_DONG_CNT'])
    관리방식 = data['resultMap_kapt']['CODE_MGR']
    복도유형 = data['resultMap_kapt']['CODE_HALL']
    난방방식 = data['resultMap_kapt']['CODE_HEAT']
    주거전용면적 = data['resultMap_kapt']['KAPT_MAREA']
    if data['resultMap_kapt_addrList'] == None:
        법정동주소 = None
    else:
        법정동주소 = data['resultMap_kapt_addrList'][0]['ADDR']
    명칭 = data['resultMap_kapt']['KAPT_NAME']
    return [kapt_code, 동수, 관리방식, 복도유형, 난방방식, 주거전용면적, 법정동주소, 명칭]

In [4]:
conn = sqlite3.connect("k-apt.db")
cur = conn.cursor()

In [ ]:
# 테이블 삭제
#cur.execute("drop table 도시")
#cur.execute("drop table 시군구")
#cur.execute("drop table 동")
#cur.execute("drop table 아파트")
#cur.execute("drop table 아파트상세")
#cur.execute("drop table 빈코드")

In [ ]:
# 테이블 생성
cur.execute("create table 도시(도시코드, 도시명)")
cur.execute("create table 시군구(시군구코드, 시군구명, 도시코드, 도시시군구코드)")
cur.execute("create table 동(동코드, 동명, 도시시군구코드, 도시시군구동코드)")
cur.execute("create table 아파트(아파트코드, 도시시군구동코드)")
cur.execute("create table 아파트상세(아파트코드, 동수, 관리방식, 복도유형, 난방방식, 주거전용면적, 법정동주소, 명칭)")

In [ ]:
cur.execute("create table 빈코드(도시시군구동코드 text)")

In [ ]:
# 도시코드 테이블 적재
data = {'도시코드':[11,26,27,28,29,30,31,36,41,42,43,44,45,46,47,48,50],
        '도시명':['서울특별시','부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도']}
cur.executemany("insert into 도시 values (?,?)", list(zip(data['도시코드'],data['도시명'])))
conn.commit()

In [ ]:
# 시군구코드 테이블 적재
cur.execute("select 도시코드 from 도시")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
for cd in codes:
    data = getAddInfo(cd,'SGG')
    data['CODE12'] = data['CODE2'].astype(str) + data['CODE'].astype(str)
    cur.executemany("insert into 시군구 values (?,?,?,?)", list(zip(data['CODE'],data['CODE_VALUE'],data['CODE2'],data['CODE12'])))
conn.commit()

In [ ]:
# 동코드 테이블 적재
cur.execute("select 도시시군구코드 from 시군구")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
for cd in codes:
    data = getAddInfo(cd,'EMD')
    data['CODE12'] = data['CODE2'].astype(str) + data['CODE'].astype(str)
    cur.executemany("insert into 동 values (?,?,?,?)",list(zip(data['CODE'],data['CODE_VALUE'],data['CODE2'],data['CODE12'])))
conn.commit()

In [ ]:
# 아파트코드 테이블 적재
cur.execute("select 도시시군구동코드 from 동 where 도시시군구동코드 not in (select distinct 도시시군구동코드 from 아파트 union select 도시시군구동코드 from 빈코드)")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
i = 0
for cd in codes:
    res = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code={}".format(cd))
    data = json.loads(res.read().decode('utf-8'))
    
    if len(data['resultList']) == 0:
        cur.execute("insert into 빈코드 values ({})".format(str(cd)))
    else:
        cur.executemany("insert into 아파트 values (?,?)",list(map(lambda x:(x['KAPT_CODE'],cd),data['resultList'])))
    i += 1
    if i % 50 == 0:
        print(list(map(lambda x:(x['KAPT_CODE'],cd),data['resultList'])))
        conn.commit()

In [10]:
cur.execute("select 아파트코드 from 아파트 where 아파트코드 not in (select 아파트코드 from 아파트상세)")
rows = cur.fetchall()
codes = list(map(lambda x:x[0],rows))
i = 0
for cd in codes:
    i += 1
    x = getAptInfo2(cd)
    cur.execute("insert into 아파트상세 values (?,?,?,?,?,?,?,?)",x)
    if i % 20 == 0:
        print(x)
        conn.commit()

In [14]:
cur.execute("select * from 아파트상세")
rows = cur.fetchall()
df = pd.DataFrame(rows)
df.head()

,0,1,2,3,4,5,6,7
0,A13593901,4,자치관리,계단식,지역난방,32275.480,서울특별시 강남구 개포동 12-2 엘지개포자이,LG개포자이아파트
1,A13593908,8,위탁관리,복도식,지역난방,46586.330,서울특별시 강남구 개포동 12 대치아파트,SH공사대치1단지
2,A13524006,13,자치관리,계단식,지역난방,75527.000,서울특별시 강남구 개포동 654 개포2차현대아파트,개포2차 현대아파트
3,A13580602,9,위탁관리,계단식,지역난방,92546.465,서울특별시 강남구 개포동 649 경남아파트,개포경남아파트
4,A13524009,11,위탁관리,복도식,지역난방,71852.900,"서울특별시 강남구 개포동 12 대치,대청 아파트",개포대치2단지


In [22]:
cur.execute("select * from 도시")
rows = cur.fetchall()
df = pd.DataFrame(rows)
df.head()

,0,1
0,11,서울특별시
1,26,부산광역시
2,27,대구광역시
3,28,인천광역시
4,29,광주광역시


In [23]:
writer = pd.ExcelWriter('result2.xlsx',engine='xlsxwriter')
df.to_excel(writer,'Sheet1')
writer.save()